In [1]:
import glob

In [6]:
OUTPUT_DIR="output"
OUTPUT_TIME="time"

EXECUTIONS=["50"]
CITIES=["13", "14", "15"]
THREADS=["1", "2", "3", "6", "9", "12"]
ITERATIONS=10

In [7]:
for exec in list(EXECUTIONS):
    for city in list(CITIES):
        file=OUTPUT_TIME + "/seq_" + city + "_cities_" + exec + "_executions_time.out"
        print(file)

time/seq_13_cities_50_executions_time.out
time/seq_14_cities_50_executions_time.out
time/seq_15_cities_50_executions_time.out
